In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [12]:
import glob
import string, os
import numpy as np
import pandas as pd
import keras.utils as ku
from numpy.random import seed
from keras.models import Sequential
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout

In [13]:
data = """The cat and her kittens
They put on their mittens,
To eat a Christmas pie.
The poor little kittens
They lost their mittens,
And then they began to cry.
O mother dear, we sadly fear
We cannot go to-day,
For we have lost our mittens."
"If it be so, ye shall not go,
For ye are naughty kittens."""

In [14]:
tokenizer = Tokenizer()

In [25]:
def dataset_preparation(data):
    corpus = data.lower().split("\n")
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1 
    input_sequences = []
    for line in corpus:
#         print(line)
        token_list = tokenizer.texts_to_sequences([line])[0]
#         print(token_list)
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
#             print("Ngram",n_gram_sequence)
            input_sequences.append(n_gram_sequence)
#     print("Input_Sequence",input_sequences)
    max_sequence_len = max([len(x) for x in input_sequences])
#     print(max_sequence_len)
    input_sequences = np.array(pad_sequences(input_sequences,   
                          maxlen=max_sequence_len, padding='pre'))
#     print(input_sequences)
#     print("\n")
#     print("\n")
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
#     print("Predictors",predictors)
#     print("Labels",label)
    label = ku.to_categorical(label, num_classes=total_words)
#     print("Transformed_Labels",label)
    return predictors,label,max_sequence_len,total_words

def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1

    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(LSTM(150))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(predictors, label, epochs=100, verbose=1)
    return model

def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
#         print(j)
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
#         print(token_list)
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
#         print(token_list)
        predicted = model.predict_classes(token_list, verbose=0)
#         print(predicted)
  
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
#                 print("Word",word)
#                 print("Index",index)
                break
        seed_text += " " + output_word
    return seed_text

In [16]:
X, Y, max_len, total_words = dataset_preparation(data)

In [17]:
model = create_model(X, Y, max_len, total_words)

Epoch 1/100
48/48 [==============================] - 3s 56ms/step - loss: 3.7616
Epoch 2/100
48/48 [==============================] - 0s 651us/step - loss: 3.7588
Epoch 3/100
48/48 [==============================] - 0s 651us/step - loss: 3.7558
Epoch 4/100
48/48 [==============================] - 0s 651us/step - loss: 3.7533
Epoch 5/100
48/48 [==============================] - 0s 651us/step - loss: 3.7507
Epoch 6/100
48/48 [==============================] - 0s 651us/step - loss: 3.7477
Epoch 7/100
48/48 [==============================] - 0s 977us/step - loss: 3.7437
Epoch 8/100
48/48 [==============================] - 0s 651us/step - loss: 3.7394
Epoch 9/100
48/48 [==============================] - 0s 651us/step - loss: 3.7320
Epoch 10/100
48/48 [==============================] - 0s 977us/step - loss: 3.7302
Epoch 11/100
48/48 [==============================] - 0s 651us/step - loss: 3.7196
Epoch 12/100
48/48 [==============================] - 0s 977us/step - loss: 3.7081
Epoch 13/100
4

48/48 [==============================] - 0s 977us/step - loss: 1.4804
Epoch 100/100
48/48 [==============================] - 0s 651us/step - loss: 1.4492


In [21]:
model

In [26]:
text = generate_text("we naughty", 3, max_len, model)

In [27]:
print(text)

we naughty lost our our
